# AutoComplete
### Aim: 
To suggest top k words given a substring
### Solution: 
- Split each word into n-grams
- Sort words based on distance between given word and each candidate. Implemented distances are
    - Jaccard distance : Intersection of ngrams over Union of ngrams
    - Intersection of ngrams : Number of ngrams common between word and candidate
- Return top k words
- Additionally, given a corpus such as a user's chat history, the k words could further be sorted based on frequency in corpus

**Importing words from corpus**
<br>Omitting words shorter than 4 letters, so that a minimum of 2 trigrams are extracted for every word.

In [1]:
from nltk import jaccard_distance,ngrams
from nltk.corpus import words
word_list = words.words()
reduced_wlist = [w.lower() for w in word_list if len(w)>4] 
print "Number of words in corpus :",len(reduced_wlist)
print "Top 10 words (alphabetically):\n",reduced_wlist[:10]

Number of words in corpus : 229483
Top 10 words (alphabetically):
[u'aalii', u'aardvark', u'aardwolf', u'aaron', u'aaronic', u'aaronical', u'aaronite', u'aaronitic', u'ababdeh', u'ababua']


**Reducing sample space for each word by filtering words that start with same substring as the word**
<br>`compare_len` : Number of characters of each word that will be matched



Note : Though this step reduces the sample space, it also assumes that the first few characters of the word have been spelled right. 
Hence it will not be able to handle typos. 
For example, if the user types *cake* instead of *bake*, the function cannot suggest *bakery*

In [2]:
def get_candidates(wlist,word,compare_len=1):
    return [w for w in wlist if w[:compare_len] == word[:compare_len]]

**Get distance between word and candidate. **
Implemented distances are
- Jaccard distance : Intersection of ngrams over Union of ngrams
- Intersection of ngrams : Number of ngrams common between word and candidate
        
<br>Note : Since Jaccard Distance will penalise longer candidates and favour shorter candidates with more ngrams common and fewer uncommon ngrams.

In [3]:
def distance(candidate,word,jacc = True):
    if jacc:
        return jaccard_distance(set(ngrams(word, n=4)), set(ngrams(candidate, n=4)))
    else:
        return len(set(ngrams(word, n=4)).intersection(set(ngrams(candidate, n=4))))*-1

**Suggest words**
<br>Get candidates and sort them by distance. Return top k candidates

In [4]:
def predict(word,k=5):
    word = word.lower()
    candidates = get_candidates(reduced_wlist,word)
    preds = sorted(candidates, key=lambda x:distance(x,word,False))[:k]
    return preds

In [5]:
for w in ['Exac','Amaz','Bril']:
    print w,":",predict(w,5)

Exac : [u'exacerbate', u'exacerbation', u'exacerbescence', u'exacerbescent', u'exact']
Amaz : [u'amaze', u'amazed', u'amazedly', u'amazedness', u'amazeful']
Bril : [u'brill', u'brilliance', u'brilliancy', u'brilliandeer', u'brilliant']


Todo
- Use a corpus such as chat history to get frequency of each word and use that as well to sort candidates
- Train a sequential model on a corpus (after embedding words) to also rank suggestions based on the words that occured before it (CBOW)
- Since one of the most common uses of AutoComplete is to fill longer words rather than shorter ones, consider ranking words by descending order of word length.